# Descriptors calculation

To run calculate descriptors using aiida-mlip you need to define some inputs as AiiDA data types, to then pass them to the calculation.

First of all we need a structure on which to perform the calculations. It will be a NaCl structure that we define using ASE, or alternatively one can choose one of the structures in the folder `Structures`.

The input structure in aiida-mlip needs to be saved as a StructureData type:

In [ ]:
from aiida import load_profile

load_profile()

In [ ]:
from aiida.orm import StructureData
from ase.build import bulk
from ase.io import read

# structure = StructureData(ase=read("Structures/qmof-ffeef76.cif"))
structure = StructureData(ase=bulk("NaCl", "rocksalt", 5.63))

Then we need to choose a model and architecture to be used for the calculation and save it as ModelData type, a specific data type of this plugin.
In this example we use MACE with a model that we download from this URL: "https://github.com/stfc/janus-core/raw/main/tests/models/mace_mp_small.model", and we save the file in the cache folder (default="~/.cache/mlips/"):


In [ ]:
from aiida_mlip.data.model import ModelData

uri = "https://github.com/stfc/janus-core/raw/main/tests/models/mace_mp_small.model"
model = ModelData.from_uri(uri, architecture="mace_mp", cache_dir="mlips")

If we already have the model saved in some folder we can save it as:

In [ ]:
# from aiida_mlip.data.model import ModelData

# model = ModelData.from_local("/path/to/model", architecture="mace")

Another parameter that we need to define as AiiDA type is the code. Assuming the code is saved as `janus` in the `localhost` computer, the code info that are needed can be loaded as follow:


In [ ]:
from aiida.orm import load_code

code = load_code("janus@localhost")

The other inputs can be set up as AiiDA Str. There is a default for every input except the structure and code. This is a list of possible inputs:

In [ ]:
from aiida.orm import Bool, Dict, Str

inputs = {
    "code": code,
    "model": model,
    "struct": structure,
    "arch": Str(model.architecture),
    "device": Str("cpu"),
    "calc_kwargs": Dict({}),
    "metadata": {"options": {"resources": {"num_machines": 1}}},
    "invariants_only": Bool(True),
    "calc_per_atom": Bool(True),
    "calc_per_element": Bool(True),
}

It's worth noting that the architecture is already defined within the model, accessible through the architecture property in the ModelData. Even if not explicitly provided as input, it will be automatically retrieved from the model.

The calculation must be set:

In [ ]:
from aiida.plugins import CalculationFactory

DescriptorsCalc = CalculationFactory("mlip.descriptors")

In this case, since we are running a descriptors calculation the entry point for the calculation is `mlip.descriptors`.
Finally, run the calculation:


In [ ]:
from aiida.engine import run_get_node

result, node = run_get_node(DescriptorsCalc, inputs)

`result` is a dictionary of the available results obtained from the calculation, while node contains the info on the node where the calculation is run:


In [ ]:
print(result)
print(node)

We can check if the calculation finished with errors. If everything worked the exit code should be 0

In [ ]:
if node.is_finished_ok:
    print(f"Calculation is finished without errors with exit status {node.exit_status}")
else:
    print(f"Some errors occurred with exit status {node.exit_status}")

If more information are needed on specific outputs they can be called like:

In [ ]:
print(result["results_dict"].get_dict())

Let's say we want the descriptor we can access it via the result or the node variable.
(when this is run in the terminal the auto-completion should help, but the idea is that the results_dict is one of the outputs which contains the main info on the calculation)

In [ ]:
print(f"Descriptor: {result['results_dict'].get_dict()['info']['mace_mp_descriptor']}")
print(f"Descriptor: {node.outputs.results_dict.get_dict()['info']['mace_mp_descriptor']}")
print(f"Cl Descriptor: {result['results_dict'].get_dict()['info']['mace_mp_Cl_descriptor']}")
print(f"Na Descriptor: {result['results_dict'].get_dict()['info']['mace_mp_Na_descriptor']}")
print(f"Descriptors: {result['results_dict'].get_dict()['mace_mp_descriptors']}")
print(node.outputs.results_dict.get_dict()["cell"][0][1])

Through the command line we can see the processes that are run

In [ ]:
! verdi process list -a

And see the results we are interested in. Substitute the number with the PK number of your calculation

In [ ]:
! verdi calcjob res {node.pk}

We can also see the inputs and outputs of the calculation

In [ ]:
! verdi node show {node.pk}